### 3. Kommunikation mit dem Netzwerk  - Teil 3: Blöcke

Neben Transaktionen müssen *FullNodes* auch Blöcke miteinander austauschen. Im Vergleich zum Transaktionsaustausch, der einseitig implementiert ist (d.h. Transaktionen können immer nur gesendet, aber nicht angefragt werden), soll der Blockaustausch beidseitig implementiert werden. Der beidseitige Austausch soll über drei Funktionen realisiert werden:

1. Blockposting **an** einen *FullNode* (dieser Teil ist eigentlich äquivalent zum Transaktionsposten an einen *FullNode*)
2. Abfrage aller bekannten Block IDs **von** einem *FullNode*
3. Abfrage eines bestimmten Blocks **von** einem *FullNode* basierend auf einer übergebenen Block ID

Durch die zusätzlichen **von** Abfragefunktionen (2. und 3.) wird einem neuen *FullNode*, der sich ins Netzwerk integrieren möchte, die Möglichkeit gegeben bei Initialisierung zuerst eine Liste aller bekannten Block IDs eines *Parent FullNodes* zu erfragen und dann die entsprechenden unbekannten Blöcke von diesem *Parent FullNode* Schritt für Schritt abzufragen.

Implementiere die drei Funktionen, indem Du zuerst drei neue Ankerpunkte in der `run_full_node.py` Datei hinzufügst:
```python

## New (4_4) run 1
@app.route("/post_block", methods=['POST'])
def post_block():
    block = Block.from_odict(create_odict_from_json(request.get_data()))
    response = full_node.process_incoming_block(block,PAUSE_MINING_EVENT)
    return json.dumps(response), 200

## New (4_4) run 2
@app.route("/get_list_of_seen_blocks", methods=['GET'])
def get_list_of_seen_blocks():
    response = full_node.get_list_of_seen_blocks()
    return json.dumps(response), 200
    
## New (4_4) run 3
@app.route("/get_block_by_id", methods=['GET'])
def get_block_by_id():
    block = full_node.get_block_by_id(request.form['block_id'])
    response = block.get_full_odict()
    return json.dumps(response), 200
    
```

* **## New (4_4) run 1**: Über den `post_block` Anker wird die bereits implementierte `process_incoming_block()` Methode der `FullNode` Klasse aufgerufen, nachdem aus den übergebenen JSON Daten (in zwei Schritten) eine `Block` Instanz erzeugt wurde. Beachte, dass beim Aufruf von `process_incoming_block` wieder ein `Event` Objekt übergeben werden muss. Durch dieses Event soll dem eventuell parallel laufenden Mining-Prozess (*wie der parallel läuft wird im nächsten Notebook erklärt*) signalisiert werden, ob es Sinn macht diesen temporär anzuhalten, bzw. neu zu starten. Z.B. macht es Sinn das Mining neu zu starten, wenn ein neuer Block zur aktuellen *Primary-Chain* des *FullNodes* hinzugefügt wurde. Sollte der *FullNode* sich dann gerade mitten im Mining befinden, hätte sein eventuell zu einem späteren Zeitpunkt gefundene Block nur sehr geringe langfristige Erfolgsaussichten. Deshalb macht ein Neustart des Minings unter Berücksichtigung des neuen Blocks, der zu *Primary-Chain* hinzugefügt wurde, in so einem Fall mehr Sinn.  
  Initialisiere zu diesem Zweck in `run_full_node.py` ein *Event*:

  ```python
  ...
  ## New (4_4) run 4
  # Initialize Event
  PAUSE_MINING_EVENT = Event()
  ...
  ```
  Das Event wird u. a. bei der Initialisierung des *FullNodes* (s. u.) verwendet. Der post_block Anker wird verwendet, wenn ein *FullNode* einen neuen Block seiner *Primary-Chain* hinzugefügt hat (siehe `process_incoming_block` Methode der `FullNode` Klasse). In diesem Fall wird der Block durch den Aufruf der folgenden `propagate_block` Methode der `FullNode` Klasse an eine zufällig ausgewählte Menge von Nachbarn weitergeleitet. Bisher war `propagate_block` nur als Dummy implementiert, erweitere nun diese Implementierung wie folgt:
  
 ```python
  ## New(4_4) 1
  def propagate_block(self, block_id):
        # Posts the transaction to randomly selected set
        # of neighbors.

        block_json  =\
        json.dumps(self.get_primary_chain().chain[block_id].get_full_odict())\
        .encode('utf8')
        selected_neighbors = self.get_neighbor_selection(2)
        if selected_neighbors:
            for n in selected_neighbors:
                result = requests.post('http://' + n + '/post_block', data = block_json)
  ```
  Den aufmerksamen Studendierenden sollte sofort auffallen, dass die Implementierung der `propagate_block` Methode in Analogie zu der schon vorgestellten Implementierung der `propagate_transaction` Methode erfolgt.
  
  
* **## New (4_4) run 2** und **## New (4_4) run 3**: Wie beschrieben, sollen hier die den Funktionsnamen entsprechenden Informationen **von** einem *FullNode* abgefragt werden. Füge hierzu die beiden Funktionen, die innerhalb der Ankerfunktionen beim *FullNode* aufgerufen werden der `FullNode` Klasse hinzu:


  ```python
    ## New(4_4) 2
    def get_list_of_seen_blocks(self):
        # Returns the list of the seen block ids
        # of the FullNode.
        
        return(self.seen_block_ids)
        
        ## New(4_4) 3
    def get_block_by_id(self, block_id):
        # Returns the block  which corresponds
        # to the block_id

        for blockchain in self.chains.values():
            if block_id in blockchain.chain.keys():
                return(blockchain.chain[block_id])
  ```

  Die Implementierungen sind selbsterklärend.  
  
  
#### Initialisierung eines FullNode

In der Regel ist ein *FullNode* nicht vom Genesis-Zeitpunkt des Systems an im Netzwerk. Ein (vereinfachtes) typisches Szenario für die Integration eines *FullNodes* ins Netzwerk ist das Folgende: Das Netzwerk besteht aus zwei *FullNodes* *A* und *B*, die beide genau die gleiche *Primary-Chain* haben. Das Netzwerk ist schon mehrere Stunden in dieser Form am Laufen, die *Primary-Chain* hat z.B. eine Länge von 25 Blöcken. Nun will ein dritter *FullNode C* sich in das Netzwerk eingliedern. Um auf den aktuellen Stand zu kommen, muss *FullNode C* zu Beginn initialisiert werden, da er ansonsten keine Möglichkeit hat an die Informationen zwischen dem aktuellen Stand der Blockchain und der im Code hinterlegten Genesis-Version der Blockchain zu kommen. Die Initialisierung erfolgt noch bevor die Flask Applikation gestartet wird. Füge hierzu folgende Zeile nach der `for` Schleife für das Hinzufügen der `neighbor` in der `run_full_node.py` Datei hinzu (innerhalb von `if neighbors is not None:`):

```python
        full_node.initialize_node(PAUSE_MINING_EVENT)
```

Dadurch wird `initialize_node` des *FullNodes* mit Übergabe des `PAUSE_MINING_EVENT` aufgerufen. Implementiere `initialize_node` (als Teil der `FullNode` Klasse) so, dass das Mining des *FullNodes* während der Initialisierung unterbrochen ist. Im Rahmen der Initialisierung soll der neue *FullNode* einfach alle ihm unbekannten Blöcke bei seinen Nachbarn abfragen und diese verarbeiten. Das kann folgendermaßen umgesetzt werden:

```python
    ## New(4_4) 4
    def initialize_node(self, pause_mining_event):
        # Initializes the FullNode by requesting all unknown
        # blocks from its neighbors.

        # Pause mining while initialization
        pause_mining_event.set()
        print('Mining interrupted for initialization.')

        # Iterate through the neighbors and get their
        # blocks to build up an own set of chains.
        for n in self.neighbors:
            # Request the seen blocks by the neighbor
            n_seen_blocks_json = requests.get\
            ('http://' + n  + '/get_list_of_seen_blocks').content
            # Make a list out of it
            n_seen_blocks = json.loads(n_seen_blocks_json)
            if len(n_seen_blocks) > 0:
                # Loop through the block ids and process the unknown ones
                for block_id in n_seen_blocks:
                    if block_id not in self.seen_block_ids:
                        # Request the content of the unknown block
                        # and process the result.
                        request_data = {'block_id' : block_id}
                        block_json = requests.get\
                        ('http://' + n  + '/get_block_by_id',\
                        data = request_data).content
                        block_odict = create_odict_from_json(block_json)
                        block = Block.from_odict(block_odict)
                        self.process_incoming_block(block,pause_mining_event)
                        
        # Restart mining
        if pause_mining_event.is_set():
            pause_mining_event.clear()
            print('Mining on the node restarted.')

```

In der `initialize_node` Implementierung wird zuerst das `pause_mining_event` gesetzt. Anschließend wird über alle Nachbarn itteriert. Von jedem Nachbar wird per Request an den `get_list_of_seen_blocks` erst die Liste aller dem Nachbar bekannten Block IDs abgefragt, bevor die Inhalte der dem initialisierten Knoten unbekannten IDs dann per Request an den `get_block_by_id` Anker des Nachbars abgefragt werden. Am Ende der Methode wir das `pause_mining_event` wieder gecleart.  
Der Aufruf der Anker und der Test der Implementierung erfolgt erst nach Einführung des Minings über Flask im nächsten Notebook, da ohne das Mining keine neuen Blöcke für das Hin- und Herschicken zur Verfügung stehen.